In [1]:
from crewai import Agent,Task,Crew,Process
from crewai_tools import SerperDevTool, Tool
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [2]:
load_dotenv()
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
SERPER_API_KEY= os.getenv("SERPER_API_KEY")

In [3]:
llm = ChatGroq(model="gemma-7b-it",groq_api_key=os.getenv("GROQ_API_KEY"))

In [4]:
# Part 1 - Create an agent in CrewAI with the role of 'Data Analyst'. The agent should have a goal of 'analyzing sales data', 
# a backstory of 'an experienced data analyst with a focus on sales trends', and a tool for 'data visualization'. 
# Include the code snippet to define this agent

from crewai_tools import tool
# Define a custom data visualization tool
@tool('data_visualization')
def data_visualization_tool(question: list) -> str:
    """ 
    This is data visualization tool which is used to visualize sales data and analyze the trends in the data.
    """
    response = llm.invoke(question)
    return response

data_visualization = data_visualization_tool

data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Analyze sales data",
    backstory="An experienced data analyst with a focus on sales trends",
    tools=[data_visualization],
    verbose=True
)

# Part 2 - Write a CrewAI task to 'collect sales data' for the 'Data Analyst' agent created in the previous part. 
# The task should include a description and expected output.
collect_sales_data_task = Task(
    description="Collect sales data for analysis",
    agent=data_analyst_agent,
    expected_output="A detailed report containing the collected sales data",
    context=[]
)

In [5]:
# Part 3 - Implement a custom tool in CrewAI that connects to a weather API and retrieves the current weather for a given location. 
# Include the code to define and register this tool.
# from crewai.tools import Tool, tool_decorator

import requests

def tool_decorator(name, description):
    def decorator(func):
        func._tool_name = name
        func._tool_description = description
        return func
    return decorator

@tool_decorator(name="WeatherTool", description="Fetches current weather for a given location")
def get_current_weather(location: str) -> str:
    API_KEY = "your-api-key"  # Replace with your actual API key
    url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={location}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()

        if 'current' in data:
            weather = data['current']['condition']['text']
            temperature = data['current']['temp_c']
            return f"The current weather in {location} is {weather} with a temperature of {temperature}°C."
        else:
            return f"Error: 'current' data not found in response for {location}. Response: {data}"
    
    except requests.RequestException as e:
        return f"Error: Unable to fetch weather data for {location}. Exception: {str(e)}"

# Custom Tool class for registering tools
class Tool:
    def __init__(self, name, func):
        self.name = name
        self.func = func

# Register the tool
weather_tool = Tool(name="WeatherTool", func=get_current_weather)


In [28]:
# Example usage of the registered tool
print(weather_tool.func("New York"))

The current weather in New York is Sunny with a temperature of 30.3°C.


In [32]:
# Example usage of the registered tool
print(weather_tool.func("Mumbai"))

The current weather in Mumbai is Mist with a temperature of 26.3°C.


In [31]:
# Example usage of the registered tool
print(weather_tool.func("Hyderabad"))

The current weather in Hyderabad is Light rain with a temperature of 23.2°C.


In [6]:
# Part 4
from crewai import Process
from crewai import Crew

# Define the Manager agent
manager_agent = Agent(
    role="Manager",
    goal="Delegate tasks to appropriate agents",
    backstory="A seasoned manager responsible for overseeing data collection and analysis",
    tools=[],
    verbose=True
)

# Create and run the crew
sales_data_crew = Crew(
    agents=[manager_agent, data_analyst_agent],
    tasks=[collect_sales_data_task],
    process=Process.hierarchical,
    manager_llm = llm,
    verbose=True
)

In [ ]:
# Running the crew
result = sales_data_crew.kickoff()
print("###################################################")
print(result)